In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [3]:
llm.invoke('how can langsmith help with testing')

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith provides a framework for creating and running automated tests. It supports various types of tests such as unit tests, integration tests, and end-to-end tests. Automated testing helps in quickly identifying issues and regressions in the codebase.\n\n2. Test Data Generation: Langsmith can generate test data to simulate various scenarios and edge cases. This is helpful in validating the behavior of the software in different scenarios and ensuring its robustness.\n\n3. Test Coverage Analysis: Langsmith can analyze the codebase and provide insights into the test coverage. It can identify areas of the code that are not adequately covered by tests, helping developers and testers prioritize their efforts in writing additional tests.\n\n4. Mocking and Stubbing: Langsmith provides tools for creating mocks and stubs, which are useful in isolating components for testing. By creating mock objects,

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are world class technical documentation writer'),
    ('user','{input}')
])

# combine to a simple LLM chain
chain = prompt | llm

# invoke and ask same question
chain.invoke({'input':'how can langsmith help with testing?'})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Test Case Management: Langsmith provides a centralized platform to manage test cases, allowing testers to create, update, and organize test cases in a structured manner. Test cases can be categorized, prioritized, and linked to requirements, making it easier to track the progress of testing activities.\n\n2. Test Execution: Langsmith allows testers to execute test cases directly from the platform, eliminating the need for manual tracking and documentation. Testers can easily record the test results, attach screenshots or logs, and add comments for each test case execution.\n\n3. Test Coverage Analysis: Langsmith provides insights into test coverage, allowing testers to track which areas of the software have been tested and which areas still need to be covered. This helps in identifying any potential gaps in testing and ensures comprehensive coverage.\n\n4. Defect Tracking: Langsmith includes a defect tracking sys

In [13]:
# add a simple output parser to convert to a string
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

text = chain.invoke({'input':'how can langsmith help with testing?'})

In [19]:
from IPython.display import HTML

def text_to_html_list(text):
    # Split the text into items based on "\n\n"
    items = text.split("\n\n")

    # Create an HTML ordered list (numbered list)
    html_list = "<ul>"

    # Add each item as a list element
    for item in items:
        html_list += f"<li>{item}</li>"

    # Close the list
    html_list += "</ol>"

    return html_list

# Your provided text
text = text
# Convert to HTML and display
HTML(text_to_html_list(text))


### Retrieval Chain

In [20]:
from langchain_community.document_loaders import WebBaseLoader
loader= WebBaseLoader('https://docs.smith.langchain.com/overview')

docs = loader.load()

In [21]:
# generate embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [22]:
# ingest documents into a vectorstore
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [23]:
# create a retrieval chain
    ## takes an incoming question
    ## looks up relevant documents
    ## passes docuemtns into an LLM to ask the question

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
# if wanted to, can run ths by passing in documents direct

from langchain_core.documents import Document

document_chain.invoke({
    'input':'How can langsmith help with testing?',
    'context': [Document(page_content='langsmith can help visualize test results')]
})

'Langsmith can help with testing by visualizing test results.'

In [25]:
# but want the documents to come from the retriever that set up

from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
# now invoke the chain
response = retrieval_chain.invoke({'input':'how can langsmith help with testing?'})
print(response['answer'])

LangSmith can help with testing by providing the following features:

1. Dataset Creation: LangSmith allows users to curate datasets easily, which can be used for testing changes to prompts or chains.

2. Running Chains: Users can run chains over the data points in a dataset and visualize the outputs. This helps in manually reviewing the results and identifying any issues or improvements.

3. Feedback and Evaluation: LangSmith enables users to assign feedback programmatically to runs. This feedback can be used to track performance over time and pinpoint underperforming data points. LangSmith also provides evaluators to assess the results of test runs and guide users to examples that require manual inspection.

4. Human Evaluation: LangSmith offers annotation queues to facilitate human review and annotation of runs. Reviewers can assess subjective qualities like creativity or humor and validate the runs that were auto-evaluated by the system.

Overall, LangSmith provides tools and funct

### Conversational Retrieval Chain
+ enable the chain to handle follow-up questions
+ retrieval method needs to take the whole history of context into account
+ the final LLM chain should also include the whole history

In [27]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

## Need a prompt to pass into LLM and query
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name='chat_history'),
    ('user','{input}'),
    ('user','Given the above conversation, generate a search query to look up in order to get information relevant to the conversation')
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [28]:
## Test by passing in instance
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    'chat_history': chat_history,
    'input': 'Tell me How'})

[Document(page_content='LangSmith Overview and User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliabl

In [29]:
# can create a new chain to continue conversation

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [30]:
# test end-to-end
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='LangSmith Overview and User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
  Document(page_content="Skip to main content\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangC

## Agent
+ an agent is where the LLM decides what steps to take

+ to start, need to decided which tools it will have acces to
  

In [31]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    'langsmith_search',
    'Search for information aobut LangSmith. For any questions about LangSmith, you must use this tool!'
)